In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import TensorDataset, random_split, DataLoader
import torch.nn as nn
import time
import wandb
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.')

Using cuda.


In [2]:
def data_loading(data_path, label_path, batch_size, train_ratio=0.8):
    data = torch.load(data_path)
    labels = torch.load(label_path)
    data = nn.functional.normalize(data, dim=1)
    
    # reshaping the dataset so  the channels will be the second dimension.
    data = data.permute(0, 2, 1)
    print(data.shape)
    n_samples, n_channels, n_features = data.shape
    X_reshaped = data.reshape(n_samples, -1) 
    # Apply SMOTE
    smote = SMOTE()
    data_resampled, labels_resampled = smote.fit_resample(X_reshaped, labels)

    # Reshape X_resampled back to 3D
    data_resampled = data_resampled.reshape(-1, n_channels, n_features)
    print(data_resampled.shape)
    data_resampled = torch.tensor(data_resampled)
    labels_resampled = torch.tensor(labels_resampled)
    dataset = TensorDataset(data_resampled, labels_resampled)
    train_size = int(train_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [3]:
def data_loading(data_path, label_path, batch_size, train_ratio=0.8):
    data = torch.load(data_path)
    labels = torch.load(label_path)
    data = nn.functional.normalize(data, dim=1)
    # print(labels.shape)
    # print(data)
    # print((labels==1).sum())
    # reshaping the dataset so  the channels will be the second dimension.
    data = data.permute(0, 2, 1)
    n_samples, n_channels, n_features = data.shape
    X_reshaped = data.reshape(n_samples, -1) 
    # Apply SMOTE
    smote = SMOTE(random_state=42, sampling_strategy='all')
    data_resampled, labels_resampled = smote.fit_resample(X_reshaped, labels)

    # Reshape X_resampled back to 3D
    data_resampled = data_resampled.reshape(-1, n_channels, n_features)
    # print(labels_resampled.shape)
    data_resampled = torch.tensor(data_resampled)
    labels_resampled = torch.tensor(labels_resampled)
    dataset = TensorDataset(data_resampled, labels_resampled)
    train_size = int(train_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=False), dataset



class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])

# Train the model
def train_lstm(train_loader, test_loader, input_dim, hidden_dim=50, output_dim=1, epochs=10, lr=0.001):
    model = LSTMModel(input_dim, hidden_dim, output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Train model
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            preds = torch.sigmoid(outputs) > 0.5
            y_true.extend(labels.numpy())
            y_pred.extend(preds.numpy())
    
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Feature Ablation: Evaluate the impact of each feature
def feature_ablation(train_loader, test_loader, num_features):
    accuracies = []
    for i in range(num_features):
        # Create subset of features
        def create_subset_loader(loader, feature_indices):
            inputs_list = []
            labels_list = []
            for inputs, labels in loader:
                inputs_list.append(inputs[:, :, feature_indices])
                labels_list.append(labels)
            dataset_subset = TensorDataset(torch.cat(inputs_list), torch.cat(labels_list))
            return DataLoader(dataset_subset, batch_size=loader.batch_size, shuffle=True)

        feature_indices = list(range(num_features))
        feature_indices.remove(i)  # Remove the i-th feature
        
        train_loader_subset = create_subset_loader(train_loader, feature_indices)
        test_loader_subset = create_subset_loader(test_loader, feature_indices)
        
        input_dim_subset = len(feature_indices)
        accuracy = train_lstm(train_loader_subset, test_loader_subset, input_dim=input_dim_subset)
        accuracies.append(accuracy)
        print(f"Feature {i} accuracy: {accuracy}")

    return accuracies

# Example usage
data_path = 'data.pt'
label_path = 'label.pt'
batch_size = 32

train_loader, test_loader, dataset = data_loading(data_path, label_path, batch_size)
num_features = dataset[0][0].shape[1]  # Number of features from the original dataset

# Perform feature ablation
accuracies = feature_ablation(train_loader, test_loader, num_features)

In [4]:
# Define the CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self, dropout):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=24, out_channels=32, kernel_size=5)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool = nn.MaxPool1d(kernel_size=2, padding=0, stride=1)
        # the size of the out channels x number  of nodes 
        self.fc1 = nn.Linear(1312, 200)
        self.fc2 = nn.Linear(200, 2)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x - self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x


In [5]:
def correct(logits, y):
    y_hat = logits.argmax(axis=1) # Finds the column with the highest value for each row of `logits`.
    return (y_hat == y).float().sum() # Computes the number of times that `y_hat` and `y` match.

In [6]:
def evaluate_metric(model, data_iter, metric):
    """Compute the average `metric` of the model on a dataset."""
    c = torch.tensor(0.).to(device)
    n = torch.tensor(0.).to(device)
    for X, y in data_iter:
        X, y = X.to(device), y.to(device) # Moves data to `device`
        logits = model(X)
        c += metric(logits, y)
        n += len(y)
    return c*100 / n

In [7]:
train, test = data_loading(data_path='NASA_data_50.pt', label_path='NASA_label_50.pt', batch_size=64)

(1370, 24, 50)


## Training

wandb: Currently logged in as: d-hadjikazemi (d-hadjikazemi-queen-mary-university-of-london). Use `wandb login --relogin` to force relogin


(1370, 24, 50)

Epoch 1/100.
Training accuracy: 50.09123992919922. Testing accuracy: 48.90510940551758. Duration: 0.277s.

Epoch 2/100.
Training accuracy: 49.63503646850586. Testing accuracy: 51.45985412597656. Duration: 0.032s.

Epoch 3/100.
Training accuracy: 49.63503646850586. Testing accuracy: 51.45985412597656. Duration: 0.029s.

Epoch 4/100.
Training accuracy: 59.215328216552734. Testing accuracy: 57.66423416137695. Duration: 0.027s.

Epoch 5/100.
Training accuracy: 82.57299041748047. Testing accuracy: 79.56204223632812. Duration: 0.026s.

Epoch 6/100.
Training accuracy: 82.02555084228516. Testing accuracy: 79.56204223632812. Duration: 0.026s.

Epoch 7/100.
Training accuracy: 83.0291976928711. Testing accuracy: 81.02189636230469. Duration: 0.025s.

Epoch 8/100.
Training accuracy: 84.21533203125. Testing accuracy: 82.11679077148438. Duration: 0.026s.

Epoch 9/100.
Training accuracy: 81.1131362915039. Testing accuracy: 81.02189636230469. Duration: 0.026s.

Epoch 10/100.
Training ac

test_acc,▁▁▇▇▇▆▇▇▇▇▇▇▇▇██▇▇█▇████████▇███████████
train_acc,▁▁▆▆▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇█▇███▇▇██████████
train_loss,██▇▅▇▆▇▆▄▄▇▂▇▃▃▆▄▃▂▁▃▃▄▁▄▄▃▁▃▁█▃▁▂▃▃▁▆▅▅
test_acc,86.49635
train_acc,97.26277
train_loss,0.52766


In [8]:
wandb.init(
    # set the wandb project where this run will be logged
    project="NASA_turbo_fan",
    # name= "with 50 data points",
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.0001,
    "architecture": "CNN",
    "epochs": 100,
    "loss": nn.BCELoss(),
    "batch_size": 64,
    "decay":1e-5,
    'dropout': 0.6
    }
)
config = wandb.config
data_path = 'NASA_data_50.pt'
label_path = 'NASA_label_50.pt'
train_loader, test_loader = data_loading(data_path=data_path, label_path=label_path,batch_size= config.batch_size)

losses = [] # Stores the loss for each training batch
train_accs = [] # Stores the training accuracy after each epoch
test_accs = [] # Stores the testing accuracy after each epoch

# Create an instance of the CNN
model = SimpleCNN(dropout=config.dropout)

# Loss and optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.decay)

model.to(device)

num_epochs = 100
for epoch in range(config.epochs):
    print(f'\nEpoch {epoch + 1}/{num_epochs}.')
    start_time = time.perf_counter()

    model.train() # This is necessary because batch normalization behaves differently between training and evaluation

    for X, y in train_loader:
        X, y = X.to(device), y.to(device) # Moves data to `device`
        logits = model(X) # Computes the logits for the batch of images `X`
        l = loss(logits, y) # Computes the loss given the `logits` and the class vector `y`
        optimizer.zero_grad() # Zeroes the gradients stored in the model parameters
        l.backward() # Computes the gradient of the loss `l` with respect to the model parameters

        optimizer.step() # Updates the model parameters based on the gradients stored inside them

        train_loss = float(l)
        losses.append(train_loss) # Stores the loss for this batch

    model.eval() # This is necessary because batch normalization behaves differently between training and evaluation
    train_acc = evaluate_metric(model, train_loader, correct)
    train_accs.append(train_acc)
    test_acc = evaluate_metric(model, test_loader, correct)
    test_accs.append(test_acc)
    metrics = {"train_loss": train_loss,
               "train_acc": train_acc,
               "test_acc": test_acc}
    wandb.log(metrics)

    end_time = time.perf_counter()

    print(f'Training accuracy: {train_accs[-1]}. Testing accuracy: {test_accs[-1]}. Duration: {end_time - start_time:.3f}s.') # Computes and displays training/testing dataset accuracy.

plt.plot(losses) # Plots the loss for each training batch
plt.xlabel('Training batch')
plt.ylabel('Cross entropy loss')
plt.show()

plt.plot(list(map(lambda x: x.cpu(),train_accs)), label='Training accuracy')
plt.plot(list(map(lambda x: x.cpu(),test_accs)), label='Testing accuracy')
plt.legend(loc='best')
plt.xlabel('Epoch')
plt.show()
wandb.finish()

In [9]:
print('max test accuracy {}'.format(max(test_accs)))

max test accuracy 87.95620727539062
